In [8]:
import quandl
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [9]:
# Define the stock ticker symbols and date range
start_date = "2015-01-01"
end_date = "2019-12-31"

tickers = ['META', 'AAPL', 'AMZN', 'NFLX', 'GOOGL']

# Define thresholds for signal generation
PE_BUY_THRESHOLD = 30
PE_SELL_THRESHOLD = 40
REVENUE_GROWTH_BUY_THRESHOLD = 0.1  # 10% growth
REVENUE_GROWTH_SELL_THRESHOLD = 0.05  # 5% growth

In [15]:
# Generate buy and sell signals
def generate_signals(financial_data):
    """
    Generate buy and sell signals based on TTM PE ratio and Revenue Growth.
    """
    signals = pd.DataFrame(index=financial_data.index)
    signals['Signal'] = 0  # Default to no signal
    
    # Buy signal: TTM PE < PE_BUY_THRESHOLD and Revenue Growth > REVENUE_GROWTH_BUY_THRESHOLD
    buy_condition = (financial_data['TTM PE'] < PE_BUY_THRESHOLD) & (financial_data['Revenue Growth'] > REVENUE_GROWTH_BUY_THRESHOLD)
    
    # Sell signal: TTM PE > PE_SELL_THRESHOLD and Revenue Growth < REVENUE_GROWTH_SELL_THRESHOLD
    sell_condition = (financial_data['TTM PE'] > PE_SELL_THRESHOLD) & (financial_data['Revenue Growth'] < REVENUE_GROWTH_SELL_THRESHOLD)
    
    signals.loc[buy_condition, 'Signal'] = 1  # Buy signal
    signals.loc[sell_condition, 'Signal'] = -1  # Sell signal
    
    return signals

In [11]:
# Backtest the strategy using historical prices
def backtest_strategy(ticker, signals, start_date="2015-01-01", end_date="2019-12-31"):
    """
    Backtest the strategy by retrieving prices and applying buy/sell signals.
    """
    # Download price data from Yahoo Finance
    price_data = yf.download(ticker, start=start_date, end=end_date)['Adj Close']
    
    # Create a DataFrame to store positions and returns
    strategy_data = pd.DataFrame(index=signals.index)
    strategy_data['Price'] = price_data.reindex(signals.index).fillna(method='ffill')
    strategy_data['Signal'] = signals['Signal']
    
    # Track positions
    strategy_data['Position'] = strategy_data['Signal'].shift().fillna(0)
    
    # Calculate returns
    strategy_data['Daily Return'] = strategy_data['Price'].pct_change()
    strategy_data['Strategy Return'] = strategy_data['Daily Return'] * strategy_data['Position']
    strategy_data['Cumulative Return'] = (1 + strategy_data['Strategy Return']).cumprod()
    
    return strategy_data


In [12]:
# Plot the cumulative return of the strategy
def plot_strategy(ticker, strategy_data):
    """
    Plot the cumulative return of the strategy.
    """
    plt.figure(figsize=(10, 6))
    plt.plot(strategy_data['Cumulative Return'], label=f'{ticker} Strategy Return')
    plt.title(f'{ticker} Strategy Cumulative Return')
    plt.xlabel('Date')
    plt.ylabel('Cumulative Return')
    plt.legend()
    plt.grid(True)
    plt.show()

In [13]:
# Main function to run the strategy for all MAANG stocks
def run_maang_strategy(tickers):
    for ticker in tickers:
        print(f'Running strategy for {ticker}')
        
        # Step 1: Retrieve financial data from Quandl
        financial_data = retrieve_financial_data(ticker)
        
        # Step 2: Generate buy/sell signals
        signals = generate_signals(financial_data)
        
        # Step 3: Backtest the strategy using historical prices
        strategy_data = backtest_strategy(ticker, signals)
        
        # Step 4: Plot the results
        plot_strategy(ticker, strategy_data)

In [14]:
# Run the strategy for all MAANG stocks
run_maang_strategy(tickers)

Running strategy for META


InvalidRequestError: (Status 422) (Quandl Error QESx08) You cannot use date column as a filter.